In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

In [16]:
df_train = pd.read_csv('train_data.csv')
df_test = pd.read_csv('test_data.csv')

In [19]:
X_train = df_train.iloc[:, :-1].values  # All columns except the last as features
y_train = df_train.iloc[:, -1].values   # Last column as target
X_test = df_test.iloc[:, 1:].values   # All columns except the first as features

In [20]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
# Define the FCNN model
model = Sequential([
    Dense(128, activation='relu', input_dim=X_train.shape[1]),  # Input layer
    Dropout(0.3),  # Regularization to avoid overfitting
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='linear')  # Output layer for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               5248      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 13569 (53.00 KB)
Trainable params: 13569 (53.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
history = model.fit(
    X_train, y_train,
    validation_split=0.2,  # Use 20% of training data for validation
    epochs=50,  # Number of epochs (adjust as needed)
    batch_size=32,  # Batch size
    verbose=1  # Display training progress
)

Epoch 1/50
3236/3236 [==============================] - 5s 1ms/step - loss: 1.8551 - mae: 0.9978 - val_loss: 0.6230 - val_mae: 0.5903
Epoch 2/50
3236/3236 [==============================] - 4s 1ms/step - loss: 1.0252 - mae: 0.7567 - val_loss: 0.5456 - val_mae: 0.5376
Epoch 3/50
3236/3236 [==============================] - 4s 1ms/step - loss: 0.8269 - mae: 0.6750 - val_loss: 0.5148 - val_mae: 0.5401
Epoch 4/50
3236/3236 [==============================] - 4s 1ms/step - loss: 0.7108 - mae: 0.6227 - val_loss: 0.4732 - val_mae: 0.5217
Epoch 5/50
3236/3236 [==============================] - 4s 1ms/step - loss: 0.6465 - mae: 0.5922 - val_loss: 0.4649 - val_mae: 0.5208
Epoch 6/50
3236/3236 [==============================] - 4s 1ms/step - loss: 0.5998 - mae: 0.5720 - val_loss: 0.5369 - val_mae: 0.5660
Epoch 7/50
3236/3236 [==============================] - 4s 1ms/step - loss: 0.5729 - mae: 0.5599 - val_loss: 0.4947 - val_mae: 0.5432
Epoch 8/50
3236/3236 [==============================] - 4s 1ms

In [21]:
y_pred = model.predict(X_test)

2016/2016 [==============================] - 1s 689us/step


In [22]:
predictions_df = pd.DataFrame(y_pred)

In [36]:
column_from_df1 = df_test["Unnamed: 0"]
column_from_df2 = predictions_df.iloc[:, -1]

In [39]:
predictions_df = pd.concat([column_from_df1, column_from_df2], axis=1)

In [43]:
predictions_df.columns = ['id', 'target_feature']

In [44]:
predictions_df

,id,target_feature
0,2016,8.316312
1,2017,8.603754
2,2018,8.830685
3,2019,8.290844
4,2020,7.353524
...,...,...
64507,193531,5.714819
64508,193532,5.347952
64509,193533,5.375154
64510,193534,5.543653


In [45]:
print(predictions_df.dtypes)

id                  int64
target_feature    float32
dtype: object


In [12]:
sample_submission = pd.read_csv("sample_submission.csv")

In [13]:
sample_submission

,id,target_feature
0,2016,5.0
1,2017,5.0
2,2018,5.0
3,2019,5.0
4,2020,5.0
...,...,...
64507,193531,5.0
64508,193532,5.0
64509,193533,5.0
64510,193534,5.0


In [46]:
print(sample_submission.dtypes)

id                  int64
target_feature    float64
dtype: object


In [47]:
# Save submission as CSV file
predictions_df.to_csv('D:\Python\Predict the wind speed at a wind turbine\\oleg_bissing_submission.csv', index=False)